# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Carnarvon,-24.8667,113.6333,64.47,94.0,40,8.05,AU,1650921858
1,Nikolskoye,59.7035,30.7861,42.87,97.0,100,5.06,RU,1650921995
2,Cayenne,4.9333,-52.3333,80.10,1.0,0,0.00,GF,1650922135
3,Plastun,44.7561,136.2939,39.31,88.0,4,10.71,RU,1650922135
4,Atuona,-9.8000,-139.0333,78.28,79.0,24,16.06,PF,1650921999
...,...,...,...,...,...,...,...,...,...
565,Tuzha,57.6062,47.9360,47.88,93.0,90,9.55,RU,1650922378
566,Nhulunbuy,-12.2333,136.7667,75.81,94.0,31,0.00,AU,1650922378
567,Prince Albert,53.2001,-105.7677,35.55,51.0,0,14.97,CA,1650922207
568,Bajos de Haina,18.4154,-70.0332,82.33,73.0,75,5.75,DO,1650922379


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
fig = gmaps.figure()

locations = cities_df[["Lat","Lng"]]

weight = cities_df["Humidity"]

heatLayer = gmaps.heatmap_layer(locations, weights=weight,dissipating=False, max_intensity=10,point_radius=1)

fig.add_layer(heatLayer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
idealCities_df = cities_df.loc[(cities_df["Max Temp"] > 70) & (cities_df["Max Temp"] < 80) & (cities_df["Wind Speed"] < 10) & (cities_df["Cloudiness"] == 0)]
idealCities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
51,Arraial do Cabo,-22.9661,-42.0278,75.06,78.0,0,6.91,BR,1650922008
117,Tura,25.5198,90.2201,72.48,83.0,0,4.74,IN,1650922173
241,Saint George,37.1041,-113.5841,78.44,11.0,0,4.61,US,1650921919
253,Vallenar,-28.5708,-70.7581,72.12,40.0,0,5.39,CL,1650922230
307,Tessalit,20.1986,1.0114,79.93,15.0,0,9.15,ML,1650922261
516,Luxor,25.6989,32.6421,78.85,17.0,0,2.30,EG,1650922358
535,Chino Valley,34.7575,-112.4538,78.42,8.0,0,8.05,US,1650922366
569,Taoudenni,22.6783,-3.9836,79.93,17.0,0,9.73,ML,1650922380


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel_df = idealCities_df[["City","Lat","Lng"]]


# lats = hotel_df["Lat"]
# lngs = hotel_df["Lng"]
# x = lats.reset_index(drop=True)
# y = lngs.reset_index(drop=True)

for i in range(0,len(x)-1):
    target_coordinates = ("-22,-42")
    target_search = "hotel"
    target_radius = 5000
    target_type = "hotel"

    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params)

    placesData = response.json()

name = placesData["results"][0]["name"]
name

'Hotel Girassol da Serra'

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# Add marker layer ontop of heat map
markerLocations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(markerLocations)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))